### halo ratio (?) function
looking at $N/N_{CDM}$ for both $z = 0$ and $z= rel.$, with mass remaining fixed and z as the independent variable. might be an interesting probe to see how scale suppression comes into play for different models.

(cont. from the w3 notebook)

In [ ]:
import os
import numpy as np
import pickle
from helpers.SimulationAnalysis import SimulationAnalysis, readHlist
import matplotlib.pylab as plt
from cycler import cycler

#......new imports (helper class)
import sys
sys.path.insert(0, '/Users/logan_main/nsbp-2023/scripts') # adding to the system path
from hlistHelper import hlist

#......formatting
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config InlineBackend.rc = {'figure.facecolor': 'w'}
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = 'cm'

In [ ]:
with open("../data/sim_data.bin", "rb") as f:
    sim_data = pickle.load(f, encoding='latin1')

#...... cold dark matter
model = 'cdm'
halo_num = 'Halo004'

cdm = hlist(PATH='../data', halo_num='', model=model)
cdm.load_hlists()
cdm.hmb = sim_data[halo_num][model][0]

#...... warm dark matter, 3 keV
model = 'wdm_3'
wdm = hlist(PATH='../data', halo_num='', model=model)
wdm.load_hlists()
wdm.hmb = sim_data[halo_num][model][0]

#### ---- original functions

In [ ]:
# TODO: come back to fix this (overflow error?)

def get_ratio(model: hlist = cdm, m: float = 0):
    a_range = [ a for a in list(model.dict.keys()) if (1.0/a) - 1.0 < 4.5]

    m = np.log10(m) # converting to log10
    ratios = np.zeros((len(a_range), ))

    halos_cdm, subhalos_cdm = model.get_z(0)
    values_cdm, cumulative_cdm, base_cdm, mass_cut_cdm = cdm.shmf(0, return_masscut_idx=True)
    mass_ind = np.argmin(np.abs(base_cdm - m))

    for i, a in enumerate(a_range):
        
        halos, subhalos = model.get_a(a)
        values, cumulative, base, mass_cut = model.shmf( (1.0/a) - 1.0, return_masscut_idx=True)
        temp = (1.*len(subhalos['Mpeak'][mass_cut])-cumulative)/(1.*len(subhalos_cdm['Mpeak'][mass_cut_cdm])-cumulative_cdm)
        ratios[i] = temp[mass_ind]

    return ratios, [(1.0/a) - 1.0 for a in a_range]

def get_ratio_relative_z(model: hlist = cdm, m: float = 0):
    a_range = [ a for a in list(model.dict.keys()) if (1.0/a) - 1.0 < 4.5]
    z_range = [(1.0/a) - 1.0 for a in a_range]

    m = np.log10(m) # converting to log10
    ratios = np.zeros((len(a_range), ))

    for i, a in enumerate(a_range):

        halos_cdm, subhalos_cdm = model.get_a(a)
        values_cdm, cumulative_cdm, base_cdm, mass_cut_cdm = cdm.shmf((1.0/a) - 1.0, return_masscut_idx=True)
        mass_ind = np.argmin(np.abs(base_cdm - m))
        
        halos, subhalos = model.get_a(a)
        values, cumulative, base, mass_cut = model.shmf( (1.0/a) - 1.0, return_masscut_idx=True)
        temp = (1.*len(subhalos['Mpeak'][mass_cut])-cumulative)/(1.*len(subhalos_cdm['Mpeak'][mass_cut_cdm])-cumulative_cdm)
        ratios[i] = temp[mass_ind]
        print('.', sep='',end='')

    return ratios, z_range
        
r_new, z_new = get_ratio_relative_z(wdm, 1e9)

#### ----- updating functions